In [ ]:
import os

print(os.getcwd())

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

os.environ["CUDA_VISIBLE_DEVICES"]="4,5" 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotCEG
import datetime
import joblib
import time

from tensorflow import keras

merge_all=pd.read_csv("/home/dp2/data/RawData_to_MereAllTable_PreprocessPipe/merge_all_20191203-20_screened.csv")
merge_all = merge_all.loc[merge_all['Time']==0]
path_indexer_df = pd.read_csv("/data/dp2/data/justin/reall/file_path_index_2023_Norm.csv")


In [ ]:
def predict_BG(pid_t_idx, implicit_HbA1c, num):
    
    sig_col = np.linspace(0,249,250)
    feat = ['HR','FW_50',"Total_Area",'HbA1C','LFP_valley', 'HFP_valley',
            'pNN20_valley','SDNN_valley','TP_valley','UT','Area50','Total_Area/HR']
    
    input_feat_test = final_df.loc[pid_t_idx][feat]
    input_feat_test['HbA1C'] = implicit_HbA1c
    input_feat_test = Scaler.transform([input_feat_test])

    input_sig_test = final_df.loc[pid_t_idx][sig_col]
    input_sig_test = np.array(input_sig_test).reshape(1,-1)

    pred_bg =reconstructed_model_avg[num].predict([input_sig_test,input_feat_test])
    
    return pred_bg

def get_pHbA1c(base_pid_t_idx):
    P_HbA1c_pack=[]
    _inconsistency=[]
    #前測Baseline 2筆資料 x3個model 

    for num in [1,2,3]:
        A_HbA1c, P_HbA1c, model_num, BG_at_6 = get_implicit_HbA1c(base_pid_t_idx,num,plot=False)
        _inconsistency.append(BG_at_6)
        P_HbA1c_pack.append(P_HbA1c)
   
    return A_HbA1c,P_HbA1c_pack,_inconsistency

def get_implicit_HbA1c(pid_t_idx, num , plot=True):
    HbA1c_tryout = np.linspace(4.5,12,76)
    sig_col = np.linspace(0,249,250)
    pred_record=[] #prediction result on range of HbA1c for each pid_t_idx
    ref_bg = final_df.loc[pid_t_idx]['BG']
    for temp_HbA1c in HbA1c_tryout:
        feat = ['HR','FW_50',"Total_Area",'HbA1C','LFP_valley', 'HFP_valley',
                'pNN20_valley','SDNN_valley','TP_valley','UT','Area50','Total_Area/HR']
        
        input_feat_base = final_df.loc[pid_t_idx][feat]
        input_feat_base['HbA1C']=temp_HbA1c
        input_feat_base = Scaler.transform([input_feat_base])

        input_sig_base = final_df.loc[pid_t_idx][sig_col]
        input_sig_base = np.array(input_sig_base).reshape(1,-1)

        pred_bg =reconstructed_model_avg[num].predict([input_sig_base,input_feat_base])
        pred_record.append(np.median(pred_bg))
        if temp_HbA1c == 6.0:
            BG_at_6 = pred_bg[0][0]

        temp_diff=[]
        for i,pred in enumerate(pred_record):
            diff = abs(pred - ref_bg)
            temp_diff.append(diff)
        i = temp_diff.index(min(temp_diff))
    if plot:
        plt.plot(final_df.iloc[0][sig_col])
        plt.show()
    #print("Actual HbA1c :{}".format(final_df.loc[final_df.index==pid_t_idx]['HbA1C'][0]))
    #print("Pseudo HbA1c :{:.1f}".format(HbA1c_tryout[i]))
    if plot:
        plt.plot(HbA1c_tryout,pred_record,label='pred_bg')
        plt.plot([0,HbA1c_tryout[i]],[ref_bg,ref_bg],ls='--',c='red',lw=2)
        plt.plot([HbA1c_tryout[i],HbA1c_tryout[i]],[0,ref_bg],ls='--',c='red',lw=2,label='ref_bg')
        plt.xlim(4.5,12)
        plt.xlabel('HbA1c %',fontsize=16)
        plt.ylim(80,350)
        plt.ylabel('BGL mg/dL',fontsize=16)
        plt.legend(loc='lower right',fontsize=16)
        plt.show()

    A_HbA1c=data_df.loc[data_df['pair']==pair]['HbA1C_base'].unique()[0]
    P_HbA1c=HbA1c_tryout[i]
    model_num=num
  
    return A_HbA1c, P_HbA1c, model_num, BG_at_6

def get_prediction(base_pid_t_idx,test_pid_t_idx,pair):

    table_dict={"person_time_idx_test":[],
          "person_time_idx_base":[],
          "implicit_HbA1c":[],
          "implicit_reject_score":[],
          "Pre_HbA1c":[],
          "Ref_HbA1c":[],
          "ref_BG":[],
          "Pred_BG_impHbA1c":[],
          'Pred_BG_refHbA1c':[],
          'Pred_BG_preHbA1c':[],
          "pred_reject_score":[],
          "pair_detail":[],
          "date_since_baseline":[]}
    
    
    full_path = path_indexer_df.loc[path_indexer_df['person_time_idx']==base_pid_t_idx]['ECGidx'].values[0]
    yymmdd = full_path.split("/")[-2]
    base_day = datetime.datetime.strptime(yymmdd,'%Y%m%d')
    
    A_HbA1c, P_HbA1c_pack,_inconsistency = get_pHbA1c(base_pid_t_idx)
    print("Actual_HbA1c",A_HbA1c)
    print("Pseudo_HbA1c_pack",P_HbA1c_pack)

    full_path = path_indexer_df.loc[path_indexer_df['person_time_idx']==test_pid_t_idx]['ECGidx'].values[0]
    yymmdd = full_path.split("/")[-2]
    test_day  = datetime.datetime.strptime(yymmdd,'%Y%m%d')
    
    delta = test_day - base_day

    pred_BG_pack=[]
    for num in [1,2,3]:
        pred_bg=predict_BG(test_pid_t_idx,np.mean(P_HbA1c_pack),num)
        pred_BG_pack.append(pred_bg[0][0])
    print("pred_bg_with_ImplicitHbA1c_pack",pred_BG_pack)

    ref_bg = data_df.loc[data_df['person_time_idx_test']==test_pid_t_idx]['BG_test'].unique()[0]
    ref_hba1c = data_df.loc[data_df['person_time_idx_test']==test_pid_t_idx]['HbA1C_test'].unique()[0]

    pred_bg_with_refHbA1c_pack=[]
    for num in [1,2,3]:
        pred_bg_with_refHbA1c=predict_BG(test_pid_t_idx,ref_hba1c,num)
        
        pred_bg_with_refHbA1c_pack.append(pred_bg_with_refHbA1c[0][0])
    print("pred_bg_with_refHbA1c_pack",pred_bg_with_refHbA1c_pack)
    
    pred_bg_with_preHbA1c_pack=[]
    for num in [1,2,3]:
        pred_bg_with_preHbA1c=predict_BG(test_pid_t_idx,A_HbA1c,num)
        pred_bg_with_preHbA1c_pack.append(pred_bg_with_preHbA1c[0][0])
    print("pred_bg_with_preHbA1c_pack",pred_bg_with_preHbA1c_pack)
    print('ref_BG',ref_bg)





    #print("Ref_BG: {}, Pred_BG: {}".format(ref_bg,pred_bg[0][0]))
    #print("=="*10)
    table_dict['person_time_idx_test'].append(test_pid_t_idx)
    table_dict['person_time_idx_base'].append(base_pid_t_idx)
    table_dict['implicit_HbA1c'].append(np.mean(P_HbA1c_pack))
    table_dict['implicit_reject_score'].append(max(_inconsistency)-min(_inconsistency)) #min max BG difference(stableness) on baseline 
    table_dict['Pre_HbA1c'].append(A_HbA1c)    
    table_dict['Ref_HbA1c'].append(ref_hba1c)
    table_dict['ref_BG'].append(ref_bg)
    table_dict['Pred_BG_impHbA1c'].append(np.median(pred_BG_pack))
    table_dict['Pred_BG_refHbA1c'].append(np.median(pred_bg_with_refHbA1c_pack))
    table_dict['Pred_BG_preHbA1c'].append(np.median(pred_bg_with_preHbA1c_pack))
    table_dict['pred_reject_score'].append(max(pred_BG_pack)-min(pred_BG_pack))
    table_dict['pair_detail'].append(pair)
    table_dict['date_since_baseline'].append(delta.days)

    #print("=="*10)

    result_table = pd.DataFrame(table_dict)        
    return result_table

In [ ]:
BL_drug_cond=(merge_all['BL_drug']==0)
BP_drug_cond=(merge_all['BP_drug']==0)
DM_drug_cond=(merge_all['DM_drug']==0)
DM_inject_cond=(merge_all['DM_inject']==0)
O_drug_cond=(merge_all['O_drug']==0)

noDM_drug_id = merge_all.loc[(merge_all['Time']==0)&(DM_drug_cond)&DM_inject_cond]['person_time_idx']  #不含糖尿藥物

DM_drug_id = merge_all.loc[(merge_all['Time']==0)&
                           BL_drug_cond&BP_drug_cond&
                           (~DM_drug_cond)&DM_inject_cond&
                           O_drug_cond]['person_time_idx']  #不含胰島素

anyDM_drug_id=merge_all.loc[(merge_all['Time']==0)&(~BL_drug_cond|~BP_drug_cond|~O_drug_cond)&
                           (~DM_drug_cond|~DM_inject_cond)]['person_time_idx'] # 有服用血糖藥 或 有胰島素注射 同時混用其他藥物


set1_target_Base_A = merge_all.loc[(merge_all['count_time']>=2)&(merge_all['count_time']<3)&
                     (merge_all['HbA1C']>6.5)&
                     (merge_all['person_time_idx'].isin(noDM_drug_id))]['count'].unique()

set1_target_Base_B = merge_all.loc[(merge_all['count_time']>=3)&
                     (merge_all['person_time_idx'].isin(noDM_drug_id))]['count'].unique()
set1_target_Base =np.concatenate([set1_target_Base_A,set1_target_Base_B])

print(set1_target_Base )

set2_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=3)&(merge_all['person_time_idx'].isin(DM_drug_id))]['count'].unique())


set3_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=3)&(merge_all['person_time_idx'].isin(anyDM_drug_id))]['count'].unique())

print("SET 1 符合條件受測者共 {} 人".format(len(set1_target_Base)))
print(set1_target_Base )

In [ ]:
final_df = pd.read_pickle('/data/dp2/data/justin/Sensor_Journal_invitation/Sensor_processedData_All.pkl')  #for implicit HbA1c
final_df=final_df.set_index('person_time_idx')
final_df.head()

In [ ]:
set1_df=pd.read_pickle("/data/dp2/data/2023_NIBG_with_FFT_input/set_1_paired_PPG_waveform+feature_extended_any2any.pkl") 

set1_df['pair'] = set1_df['person_time_idx_base']+'_'+set1_df['person_time_idx_test']

set1_target = merge_all.loc[merge_all['count'].isin(set1_target_Base)|merge_all['Person No'].isin(set1_target_Base)]['person_time_idx']

set1_df = set1_df.loc[(set1_df['person_time_idx_base'].isin(set1_target))|set1_df['person_time_idx_test'].isin(set1_target)]

model_name ='2023_HbA1c_noProfile_pseudoHbA1c_noDMdrug_4090'

Scaler = joblib.load("/data/dp2/data/justin/Sensor_Journal_invitation/{}_Scaler.save".format(model_name))  

reconstructed_model_avg={}
pid_record={}
for num in [1,2,3]:
    reconstructed_model_avg[num] =keras.models.load_model("/data/dp2/data/justin/Sensor_Journal_invitation/saved_models/{}_{}.h5".format(model_name,num), compile=False )

    reconstructed_model_avg[num].compile(loss='LogCosh', optimizer='adam',run_eagerly=True)

    start = time.time() # Mark start trining time

In [ ]:
output_df_hist=pd.DataFrame()

data_df=set1_df

#base on the pair of data in data_df and use waveform data in final_df
for pair in data_df['pair'].unique():
    base_pid = pair.split('_')[0]+'_'+pair.split('_')[1]
    test_pid = pair.split('_')[2]+'_'+pair.split('_')[3]
    print(base_pid,test_pid)
    result_df = get_prediction(base_pid,test_pid,pair)
    
    output_df_hist = pd.concat([output_df_hist,result_df])
    



In [ ]:
output_df_hist.to_pickle("2024_ImplicitHbA1c_set1_extended_any2any.pkl")